In [1]:
import torch
#import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import timeit
from tqdm import tqdm
import math


In [2]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F
import re

from datasets import load_dataset



text_data = load_dataset("yahoo_answers_topics") #load_dataset("ag_news")

def prepare_data(data_point):
    data_point['text'] = data_point['question_title'] + " " + data_point['question_content']
    return data_point

text_data = text_data.map(prepare_data)
text_data = text_data.rename_column("topic", "label")

sequences = text_data['train']['text']
labels=text_data['train']['label']

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import RobertaTokenizer
# Initialize tokenizer
#seq_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
seq_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# Define the Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        assert len(texts) == len(labels), "Texts and labels must have the same length."
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and encode the input text
        encoded = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        # Add the label to the item
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


# Dataset and DataLoader creation

MAX_WORDS=18
length=len(sequences[0:5000])
trainset = TextDataset(
        texts=sequences[0:length],
        labels=labels[0:length],
        tokenizer=seq_tokenizer,
        max_length=MAX_WORDS  # Modify if needed based on your text length
    )
trainloader = DataLoader(trainset, batch_size=48, shuffle=True)
torch.manual_seed(42)
    # Test a batch
index = iter(trainloader)
batch = next(index)
#print(batch)  # View the first batch


In [4]:
from transformers import RobertaTokenizer


input_idx = batch['input_ids']
attention_masks = batch['attention_mask']  #
labels = batch['labels']  # Labels tensor

decoded = seq_tokenizer.batch_decode(input_idx, skip_special_tokens=False)

tokenized_sequences = [seq_tokenizer.convert_ids_to_tokens(ids) for ids in input_idx]

    # Print the tokenized sentences
# for i, tok in enumerate(tokenized_sequences):
#     print(f"Sequence {i + 1} Tokens:")
#     print(tok)


In [5]:
RANDOM_SEED = 42
EPOCHS =  1 #40
LEARNING_RATE = 1e-4
NUM_CLASSES = 10

global GLOBAL_VAR
GLOBAL_VAR=0
DROPOUT = 0 # 0.001
#HIDDEN_DIM = 768
ADAM_WEIGHT_DECAY = 0
ADAM_BETAS = (0.9, 0.999)
#ACTIVATION="gelu"
NUM_ENCODERS = 1

NUM_PATCHES = MAX_WORDS

EMBED_DIM = 768
NUM_HEADS = 12 #NUM_PATCHES+1
HEAD_DIM= EMBED_DIM//NUM_HEADS
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cpu" if torch.cuda.is_available() else "cpu"

In [6]:
from transformers import RobertaModel, BertModel
import torch
# Load the pretrained RoBERTa model
model_pretrained = RobertaModel.from_pretrained("roberta-base")

# Extract the word embeddings from the pretrained model
embeddings = model_pretrained.get_input_embeddings().weight.detach().cpu()

# print(embeddings.shape)
# model_pretrained = BertModel.from_pretrained("bert-base-uncased")

# # Extract the word embeddings from the pretrained model
# embeddings = model_pretrained.get_input_embeddings().weight.detach().cpu()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#from Roberta_arch import RoBERTa
from roberta_module import RoBERTa
r=4#125
#model = ViT(r,EMBED_DIM, PATCH_DIM, PATCH_SIZE, NUM_PATCHES,NUM_HEADS, HEAD_DIM, DROPOUT, CHANNELS, NUM_CLASSES).to(device)
model = RoBERTa(r, embedding_dim=768, vocab_size=embeddings.shape[0], max_seq_len=MAX_WORDS, num_head=NUM_HEADS, dim_head=HEAD_DIM, dropout=0, num_classes=NUM_CLASSES)
# Disable gradients for the embedding layer in the BERT model
for param in model.embedding.word_embeddings.parameters():
    param.requires_grad = False

#print(model(input_idx))


In [8]:
import copy
w_glob=model.state_dict()
#Use the pretrained word_embedding without any malicious modification
w_glob['embedding.word_embeddings.weight']=copy.deepcopy(embeddings)
#print(w_glob.keys())
D=EMBED_DIM

In [9]:
#Design the parameters
c=10**2
c2=3
target_encoders=range(1,5)
from Design_Model_LORA import Design

All_PATCH=NUM_PATCHES+1
ENCODER_NO=12


malicious_model=Design(r, c, c2, EMBED_DIM,EMBED_DIM,NUM_HEADS, NUM_PATCHES)

w_glob['embedding.position_embeddings']=malicious_model.position_encoding(w_glob,HEAD_DIM)


std1=torch.std(w_glob['embedding.position_embeddings'][0][2])
mu1=torch.mean(w_glob['embedding.position_embeddings'][0][2])

w_glob['embedding.LN.weight']=std1*torch.ones(w_glob['embedding.LN.weight'].size())
w_glob['embedding.LN.bias']=mu1*torch.ones(w_glob['embedding.LN.bias'].size())
patch_id=range(1,r*len(target_encoders)+1)

for i in range(1,ENCODER_NO+1):
  key01=f'encoder{i}.attn.Bq.weight'
  w_glob[key01]=torch.zeros(w_glob[key01].size())

  key02=f'encoder{i}.attn.Bk.weight'
  w_glob[key02]=torch.zeros(w_glob[key02].size())

  key03=f'encoder{i}.attn.Bv.weight'
  w_glob[key03]=torch.zeros(w_glob[key03].size())

  key04=f'encoder{i}.attn.Aq.weight'
  w_glob[key04]=torch.zeros(w_glob[key04].size())

  key05=f'encoder{i}.attn.Ak.weight'
  w_glob[key05]=torch.zeros(w_glob[key05].size())

  key06=f'encoder{i}.attn.Av.weight'
  w_glob[key06]=torch.zeros(w_glob[key06].size())

for i in range(1,ENCODER_NO): #ENCODER_NO):

    w_glob[f'encoder{i}.attn.QKV.weight']=malicious_model.attention(w_glob[f'encoder{i}.attn.QKV.weight'])
    w_glob[f'encoder{i}.attn.QKV.bias']=torch.zeros(w_glob[f'encoder{i}.attn.QKV.bias'].size())


    key1=f'encoder{i}.mlp.fc1.weight'
    w_glob[key1]=torch.zeros(w_glob[key1].size())

    key2=f'encoder{i}.mlp.fc2.weight'
    w_glob[key2]=torch.zeros(w_glob[key2].size())

    key3=f'encoder{i}.mlp.fc1.bias'
    w_glob[key3]=torch.zeros(w_glob[key3].size())
    #w_glob[key3][0:EMBED_DIM]=high*torch.ones(w_glob[key3][0:EMBED_DIM].size())

    key4=f'encoder{i}.mlp.fc2.bias'
    #w_glob[key4]=torch.zeros(w_glob[key4].size())
    w_glob[key4]=torch.zeros(w_glob[key4].size())

    key5=f'encoder{i}.attn.msa.weight'
    w_glob[key5]=torch.zeros(w_glob[key5].size())

    key6=f'encoder{i}.attn.A.weight'
    w_glob[key6]=torch.zeros(w_glob[key6].size())

    key7=f'encoder{i}.attn.B.weight'
    w_glob[key7]=torch.zeros(w_glob[key7].size())

    key8=f'encoder{i}.attn.LN1.weight'
    w_glob[key8]=std1*torch.ones(w_glob[key8].size())

    key9=f'encoder{i}.attn.LN1.bias'
    w_glob[key9]=mu1*torch.ones(w_glob[key9].size())

    key10=f'encoder{i}.mlp.LN2.weight'
    w_glob[key10]=std1*torch.ones(w_glob[key10].size())

    key11=f'encoder{i}.mlp.LN2.bias'
    w_glob[key11]=mu1*torch.ones(w_glob[key11].size())


count=0
for target_encoder in target_encoders:
    for i in range(r):

        w_glob[f'encoder{target_encoder}.attn.A.weight'][i][2*patch_id[count+i]+1]=1

        w_glob[f'encoder{target_encoder}.attn.B.weight']=torch.zeros(w_glob['encoder11.attn.B.weight'].size())

    count+=r

#Encoder 12
w_glob['encoder12.attn.QKV.weight']=torch.zeros(w_glob['encoder11.attn.QKV.weight'].size())
w_glob['encoder12.attn.QKV.bias']=torch.zeros(w_glob['encoder11.attn.QKV.bias'].size())
w_glob['encoder12.attn.QKV.weight'][2*EMBED_DIM:3*EMBED_DIM]=torch.eye(EMBED_DIM) #Set value matrix to identity
w_glob['encoder12.attn.msa.weight']=torch.eye(EMBED_DIM)


a=torch.zeros(EMBED_DIM)
for i in range(NUM_PATCHES+1):
    a[2*i]=c/(NUM_PATCHES+1)
    a[2*i+1]=-c/(NUM_PATCHES+1)
#print(a)
for h in range(1, NUM_HEADS):
    for i in range(NUM_PATCHES+1):
        if i<31:
            a[h*HEAD_DIM+2*i]=c2/(NUM_PATCHES+1)
            a[h*HEAD_DIM+2*i+1]=-c2/(NUM_PATCHES+1)
        else:
            a[h*HEAD_DIM+2*31]=c2/(NUM_PATCHES+1)
            a[h*HEAD_DIM+2*31+1]=-c2/(NUM_PATCHES+1)

a[0]+=c
a[1]+=-c
for h in range(1, NUM_HEADS):
    a[h*HEAD_DIM+0]+=c2
    a[h*HEAD_DIM+1]+=c2

std2=torch.std(a)
mu2=torch.mean(a)

w_glob['encoder12.attn.LN1.weight']=std2*torch.ones(w_glob['encoder11.attn.LN1.weight'].size())
w_glob['encoder12.attn.LN1.bias']=mu2*torch.ones(w_glob['encoder11.attn.LN1.weight'].size())

w_glob['encoder12.attn.A.weight']=torch.zeros(w_glob['encoder12.attn.A.weight'].size())
w_glob['encoder12.attn.B.weight']=torch.zeros(w_glob['encoder12.attn.B.weight'].size())

w_glob['encoder12.mlp.LN2.weight']=std2*torch.ones(w_glob['encoder11.attn.LN1.weight'].size())
w_glob['encoder12.mlp.LN2.bias']=mu2*torch.ones(w_glob['encoder11.attn.LN1.weight'].size())

w_glob['encoder12.mlp.fc1.weight']=torch.zeros(w_glob['encoder12.mlp.fc1.weight'].size())

w_glob['encoder12.mlp.fc1.bias']=torch.zeros(w_glob['encoder12.mlp.fc1.bias'].size())

w_glob['encoder12.mlp.fc2.weight']=torch.zeros(w_glob['encoder12.mlp.fc2.weight'].size())

w_glob['encoder12.mlp.fc2.bias']=torch.zeros(w_glob['encoder12.mlp.fc2.bias'].size())



w_glob['mlphead.dense.weight'] = torch.eye(EMBED_DIM)
w_glob['mlphead.dense.bias'] = torch.zeros(EMBED_DIM)
w_glob['mlphead.head.weight']=torch.zeros(w_glob['mlphead.head.weight'].size())
w_CLS=10**2
for i in range(len(patch_id)):
    w_glob['mlphead.head.weight'][0][2*patch_id[i]]=w_CLS
w_glob['mlphead.head.bias']=torch.zeros(w_glob['mlphead.head.bias'].size())

In [10]:
GLOBAL_VAR=0
model.load_state_dict(w_glob)

<All keys matched successfully>

In [12]:
from User_Train import train
sample=23
#target_encoder=8
weight_grad=train(target_encoders,sample, model,input_idx, labels)
#print(weight_grad)

torch.Size([18])


In [14]:
weight=w_CLS#*w_v
LN_scaling= 1 #2*2 #coz divied by 2 two times
div_factor=LN_scaling*(NUM_PATCHES+1)
a=c**2
b=std1**2*D
new_scaling=a/b
noise_std=0 #0.1
from reconstruction import recover1, recover2


for u in range(1,len(target_encoders)+1):
    print(f'encoder:{u} words')
    recovered_embed=recover1(noise_std, patch_id[r*(u-1):r*u], w_glob, weight_grad[u-1].unsqueeze(0),weight, div_factor, new_scaling,D)
    recovered_words=recover2(patch_id[r*(u-1):r*u], recovered_embed, w_glob, seq_tokenizer)



encoder:1 words
Recovered embedding 1 corresponds to the word: Which
Recovered embedding 2 corresponds to the word: Ġk
Recovered embedding 3 corresponds to the word: enn
Recovered embedding 4 corresponds to the word: edy
encoder:2 words
Recovered embedding 1 corresponds to the word: Ġdied
Recovered embedding 2 corresponds to the word: Ġin
Recovered embedding 3 corresponds to the word: Ġa
Recovered embedding 4 corresponds to the word: Ġskiing
encoder:3 words
Recovered embedding 1 corresponds to the word: Ġacc
Recovered embedding 2 corresponds to the word: ed
Recovered embedding 3 corresponds to the word: ent
Recovered embedding 4 corresponds to the word: ?
encoder:4 words
Recovered embedding 1 corresponds to the word: Ġ
Recovered embedding 2 corresponds to the word: </s>
Recovered embedding 3 corresponds to the word: <pad>
Recovered embedding 4 corresponds to the word: <pad>


In [15]:
vocab = seq_tokenizer.get_vocab()
index_to_token = {index: token for token, index in vocab.items()}
for t in range(0,len(patch_id)):
    ground_truth=[input_idx[sample][patch_id[t]]]
    words = [index_to_token[idx.item()] if isinstance(idx, torch.Tensor) else index_to_token[idx] for idx in ground_truth]
# Print the results
    for idx, word in enumerate(words):
        print(f"ground truth {t+1} corresponds to the word: {word}")

ground truth 1 corresponds to the word: Which
ground truth 2 corresponds to the word: Ġk
ground truth 3 corresponds to the word: enn
ground truth 4 corresponds to the word: edy
ground truth 5 corresponds to the word: Ġdied
ground truth 6 corresponds to the word: Ġin
ground truth 7 corresponds to the word: Ġa
ground truth 8 corresponds to the word: Ġskiing
ground truth 9 corresponds to the word: Ġacc
ground truth 10 corresponds to the word: ed
ground truth 11 corresponds to the word: ent
ground truth 12 corresponds to the word: ?
ground truth 13 corresponds to the word: Ġ
ground truth 14 corresponds to the word: </s>
ground truth 15 corresponds to the word: <pad>
ground truth 16 corresponds to the word: <pad>
